In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 4


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-06 11:05:56] ax.service.ax_client: Generated new trial 64 with parameters {'Drug': 93, 'SL_1': 71, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 0, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the rel

11 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,0,0
1,0,0,0,0
2,2,0,0,0
3,0,0,1,0
4,0,0,0,0
5,0,0,0,0
6,0,0,99,0
7,0,0,0,0
8,1,0,0,0
9,0,0,28,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
0            64    93    71     0     0     0     0     0    0    0  ...    0   
1            65    93    71     0     0     0     0     0    0    0  ...    0   
4            68    93    71     0     0     0     0     0    0    0  ...    0   
5            69    93    71     0     0     0     0     0    0    0  ...    0   
7            71    93    71     0     0     0     0     0    0    0  ...    0   
10           74    94    34     0     0     0     0     0    0    0  ...    0   
11           75    93    71     0     0     0     0     0    0    0  ...    0   
12           76    93    71     0     0     0     0     0    0    0  ...    0   
13           77    93    71     0     0     0     0     0    0    0  ...    0   
14           78    93    71     0     0     0     0     0    0    0  ...    0   
15           79    94    33     0     0     0     0     0    0    0  ...    0   

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,0,100
1,0,0,0,100
2,2,0,0,0
3,0,0,1,0
4,0,0,0,100
5,0,0,0,100
6,0,0,99,0
7,0,0,0,100
8,1,0,0,0
9,0,0,28,0


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000
mean,71.500000,0.590125,0.409875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0,0.187500,0.687500
std,4.760952,0.057579,0.057579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.341565,0.0,0.403113,0.478714
min,64.000000,0.567000,0.260000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
25%,67.750000,0.567000,0.430000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
50%,71.500000,0.567000,0.433000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
75%,75.250000,0.570000,0.433000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
max,79.000000,0.740000,0.433000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000
mean,71.500000,93.937500,66.375000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187500,0.0,8.000000,68.750000
std,4.760952,2.205108,12.836796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.543906,0.0,25.248102,47.871355
min,64.000000,93.000000,33.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
25%,67.750000,93.000000,71.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
50%,71.500000,93.000000,71.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,100.000000
75%,75.250000,93.250000,71.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,100.000000
max,79.000000,100.000000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,99.000000,100.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000,16.0
mean,71.500000,206.543750,143.456250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.000000,0.0,187.500000,687.500000,0.0
std,4.760952,20.152575,20.152575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.565026,0.0,403.112887,478.713554,0.0
min,64.000000,198.450000,91.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
25%,67.750000,198.450000,150.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
50%,71.500000,198.450000,151.550000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1000.000000,0.0
75%,75.250000,199.500000,151.550000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1000.000000,0.0
max,79.000000,259.000000,151.550000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.000000,0.0,1000.000000,1000.000000,0.0


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        3.3
SL_1        2.3
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.0
LL_3        0.0
P_1         0.0
P_2         0.0
P_3         0.0
S_1         2.0
S_2         0.0
S_3         3.0
Water      11.0
Solvent     0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_4_OT_2_protocol_1.py',
 'protocol/iteration_4_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,64,0.567,0.433,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,65,0.567,0.433,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,66,0.579,0.421,0,0,0,0,0,0,0,0,1,0,0,0,2,0
3,67,0.567,0.433,0,0,0,0,0,0,0,0,0,0,1,0,2,0
4,68,0.567,0.433,0,0,0,0,0,0,0,0,0,0,0,1,1,0
5,69,0.567,0.433,0,0,0,0,0,0,0,0,0,0,0,1,1,0
6,70,0.567,0.433,0,0,0,0,0,0,0,0,0,0,1,0,2,0
7,71,0.567,0.433,0,0,0,0,0,0,0,0,0,0,0,1,1,0
8,72,0.585,0.415,0,0,0,0,0,0,0,0,1,0,0,0,2,0
9,73,0.567,0.433,0,0,0,0,0,0,0,0,0,0,1,0,2,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,267.401331,0.228684
1,Caution,A2,267.880354,0.332988
2,Caution,A3,252.382602,0.103058
3,Good,A4,247.880060,0.209892
4,Good,A5,247.449566,0.157954
5,Good,A6,246.122520,0.163523
6,Good,B1,260.288601,0.186036
7,Good,B2,267.453358,0.219455
8,Good,B3,256.159877,0.166164
9,Good,B4,246.826891,0.271515


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,247.150715,0.916090,0.177123,0.028515,1
2,Formulation 3,261.300612,5.714351,0.190552,0.026931,1
3,Formulation 4,240.985371,8.261157,0.278453,0.009812,1
4,Formulation 5,245.920704,10.793356,0.193196,0.027674,1
5,Formulation 6,260.964197,6.395231,0.214088,0.030592,1
6,Formulation 7,243.512584,3.288847,0.406869,0.048067,1
7,Formulation 8,253.380967,14.075550,0.209159,0.045591,1
8,Formulation 9,269.600234,8.841119,0.264390,0.097252,1
9,Formulation 10,256.723385,9.923296,0.367061,0.040796,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,64,0.567,0.433,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,65,0.567,0.433,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 2,247.150715,0.916090,0.177123,0.028515,1
2,66,0.579,0.421,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 3,261.300612,5.714351,0.190552,0.026931,1
3,67,0.567,0.433,0,0,0,0,0,0,0,...,1,0,2,0,Formulation 4,240.985371,8.261157,0.278453,0.009812,1
4,68,0.567,0.433,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 5,245.920704,10.793356,0.193196,0.027674,1
5,69,0.567,0.433,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 6,260.964197,6.395231,0.214088,0.030592,1
6,70,0.567,0.433,0,0,0,0,0,0,0,...,1,0,2,0,Formulation 7,243.512584,3.288847,0.406869,0.048067,1
7,71,0.567,0.433,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 8,253.380967,14.075550,0.209159,0.045591,1
8,72,0.585,0.415,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 9,269.600234,8.841119,0.264390,0.097252,1
9,73,0.567,0.433,0,0,0,0,0,0,0,...,1,0,2,0,Formulation 10,256.723385,9.923296,0.367061,0.040796,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,64,0.567,0.433,0,0,0,0,0,0,0,...,1,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,65,0.567,0.433,0,0,0,0,0,0,0,...,1,0,Formulation 2,247.150715,0.916090,0.177123,0.028515,1,1134.0,0
2,66,0.579,0.421,0,0,0,0,0,0,0,...,2,0,Formulation 3,261.300612,5.714351,0.190552,0.026931,1,1158.0,0
3,67,0.567,0.433,0,0,0,0,0,0,0,...,2,0,Formulation 4,240.985371,8.261157,0.278453,0.009812,1,1134.0,0
4,68,0.567,0.433,0,0,0,0,0,0,0,...,1,0,Formulation 5,245.920704,10.793356,0.193196,0.027674,1,1134.0,0
5,69,0.567,0.433,0,0,0,0,0,0,0,...,1,0,Formulation 6,260.964197,6.395231,0.214088,0.030592,1,1134.0,0
6,70,0.567,0.433,0,0,0,0,0,0,0,...,2,0,Formulation 7,243.512584,3.288847,0.406869,0.048067,1,1134.0,0
7,71,0.567,0.433,0,0,0,0,0,0,0,...,1,0,Formulation 8,253.380967,14.075550,0.209159,0.045591,1,1134.0,0
8,72,0.585,0.415,0,0,0,0,0,0,0,...,2,0,Formulation 9,269.600234,8.841119,0.264390,0.097252,1,1170.0,0
9,73,0.567,0.433,0,0,0,0,0,0,0,...,2,0,Formulation 10,256.723385,9.923296,0.367061,0.040796,1,1134.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
1,1,0.247151,0.000916,0.567,0.0,0.177123,0.028515,0.083333
2,1,0.261301,0.005714,0.579,0.0,0.190552,0.026931,0.166667
3,1,0.240985,0.008261,0.567,0.0,0.278453,0.009812,0.166667
4,1,0.245921,0.010793,0.567,0.0,0.193196,0.027674,0.083333
5,1,0.260964,0.006395,0.567,0.0,0.214088,0.030592,0.083333
6,1,0.243513,0.003289,0.567,0.0,0.406869,0.048067,0.166667
7,1,0.253381,0.014076,0.567,0.0,0.209159,0.045591,0.083333
8,1,0.269600,0.008841,0.585,0.0,0.264390,0.097252,0.166667
9,1,0.256723,0.009923,0.567,0.0,0.367061,0.040796,0.166667


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
1,1,0.247151,0.000916,0.567,0.0,0.177123,0.028515,0.083333
2,1,0.261301,0.005714,0.579,0.0,0.190552,0.026931,0.166667
3,1,0.240985,0.008261,0.567,0.0,0.278453,0.009812,0.166667
4,1,0.245921,0.010793,0.567,0.0,0.193196,0.027674,0.083333
5,1,0.260964,0.006395,0.567,0.0,0.214088,0.030592,0.083333
6,1,0.243513,0.003289,0.567,0.0,0.406869,0.048067,0.166667
7,1,0.253381,0.014076,0.567,0.0,0.209159,0.045591,0.083333
8,1,0.269600,0.008841,0.585,0.0,0.264390,0.097252,0.166667
9,1,0.256723,0.009923,0.567,0.0,0.367061,0.040796,0.166667


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,64,0.567,0.433,0,0,0,0,0,0,0,...,0.083333,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,65,0.567,0.433,0,0,0,0,0,0,0,...,0.083333,0,Formulation 2,0.247151,0.000916,0.177123,0.028515,1,0.567,0
2,66,0.579,0.421,0,0,0,0,0,0,0,...,0.166667,0,Formulation 3,0.261301,0.005714,0.190552,0.026931,1,0.579,0
3,67,0.567,0.433,0,0,0,0,0,0,0,...,0.166667,0,Formulation 4,0.240985,0.008261,0.278453,0.009812,1,0.567,0
4,68,0.567,0.433,0,0,0,0,0,0,0,...,0.083333,0,Formulation 5,0.245921,0.010793,0.193196,0.027674,1,0.567,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-06 13:17:48] ax.service.ax_client: Completed trial 64 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-06 13:17:48] ax.service.ax_client: Completed trial 65 with data: {'Solu': (0.567, 0), 'Size': (0.247151, 0.000916), 'PDI': (0.177123, 0.028515)}.
[INFO 08-06 13:17:48] ax.service.ax_client: Completed trial 66 with data: {'Solu': (0.579, 0), 'Size': (0.261301, 0.005714), 'PDI': (0.190552, 0.026931)}.
[INFO 08-06 13:17:48] ax.service.ax_client: Completed trial 67 with data: {'Solu': (0.567, 0), 'Size': (0.240985, 0.008261), 'PDI': (0.278453, 0.009812)}.
[INFO 08-06 13:17:48] ax.service.ax_client: Completed trial 68 with data: {'Solu': (0.567, 0), 'Size': (0.245921, 0.010793), 'PDI': (0.193196, 0.027674)}.
[INFO 08-06 13:17:48] ax.service.ax_client: Completed trial 69 with data: {'Solu': (0.567, 0), 'Size': (0.260964, 0.006395), 'PDI': (0.214088, 0.030592)}.
[INFO 08-06 13:17:48] ax.service.ax_client: Completed trial 70 with data: {'Solu': (0.567, 0